# Tradier API

Obtain Daily Historical Data with Tradier's Developer Sandbox API https://documentation.tradier.com/brokerage-api

Code from https://github.com/rebekah999/Tradier-API repository by Rebekah Fowler

In [1]:
# basic imports for overall functionality
import numpy as np 
import pandas as pd
import requests
import ast
import http.client as httplib
from datetime import timedelta

# Error handling
import traceback

#Saving list a file
import pickle

## API Class

In [2]:
class API():
    
    def __init__(self, portfolio, key):
        self.portfolio = [p.upper() for p in list(sorted(set(portfolio)))]
        self.key = key
        
        self.start = str(pd.datetime.today() - timedelta(days=100)).split(" ")[0] #days can be changed to retrieve more or less data
        self.end = str(pd.datetime.today() + timedelta(days=1)).split(" ")[0]
        self.now = str(pd.datetime.today()).split(" ")[0]
        
        self.loop_port = self.loop_port(self.portfolio, self.start, self.end, self.key)
        
        
    
    #API 
    def tradier(self, key, endpoint, par):
        connection = httplib.HTTPSConnection('sandbox.tradier.com', 443, timeout = 30)
        # Headers

        headers = {"Accept":"application/json",
               "Authorization":"Bearer {}".format(key)}

        # Send synchronously
        connection.request('GET', '{}{}'.format(endpoint, par), None, headers, encode_chunked=False)
        try:
          response = connection.getresponse()
          content = response.read()
          # Success
          print('Response status ' + str(response.status))
          #print(response.headers)

        except httplib.HTTPException:
            # Exception
            print('Exception during request')
        return content

    def b_decode(self, content):
        new_str = content.decode('utf-8') # Decode using the utf-8 encoding
        save = ast.literal_eval(new_str)
        col1 = [col for col in save.keys()][0]
        cut_down = save[col1]

        col2 = [col for col in cut_down.keys()][0]
        cut_down2 = cut_down[col2]

        df = pd.DataFrame(cut_down2)
        df.columns = df.columns.str.capitalize()
        return df
    
    def loop_port(self, portfolio, start, end, key):
        port = pd.DataFrame(columns=['Close', 'Date', 'High', 'Low', 'Open', 'Volume', 'Symbol'])
        slist = []
        for symbol in portfolio:

            content = self.tradier(key, '/v1/markets/history',
                      "?symbol={}&session_filter=open&start={}&end={}".format(symbol, start, end))
            try:
                file = pd.DataFrame(self.b_decode(content))
                file["Symbol"] = symbol
                slist.append(file)
                print(symbol)
            except:
                print(content, symbol)

        return slist

# Inputs

In [3]:
#List of portfolio stocks does not have to be capitalized
portfolio = ["SPY", "DIA", "AAPL"] # <-- Add more symbols here


key = "Your-API-Key" # Tradier API Key goes here

# Save to csv

In [4]:
api = API(portfolio, key) #a variable is standard for some functions to work

Response status 200
AAPL
Response status 200
DIA
Response status 200
SPY


In [5]:
portfolio_list = api.loop_port

In [6]:
portfolio_list[0].tail()

,Close,Date,High,Low,Open,Volume,Symbol
66,309.54,2020-05-14,309.79,301.5300,304.51,39732269,AAPL
67,307.71,2020-05-15,307.90,300.2100,300.35,41587094,AAPL
68,314.96,2020-05-18,316.50,310.3241,313.17,33843125,AAPL
69,313.14,2020-05-19,318.52,313.0100,315.03,25432385,AAPL
70,319.23,2020-05-20,319.52,316.2000,316.68,27850874,AAPL


### To save data into a file

In [7]:
with open('Tradier_API.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(portfolio_list, filehandle)
    filehandle.close()

### Use to code below to open saved pickle file in another enviroment or use a.loop_port in this enviroment

In [11]:
#You can open saved pickel file like this 
with open("Tradier_API.data", 'rb') as filehandle:
        API_DATA = pickle.load(filehandle) # API_DATA == portfolio_list